In [1]:
import jinja2
from  anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, typespec

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that always responds with a greeting."
application_description = "Bot that writes poems about cars."

In [4]:
content = jinja_env.from_string(typespec.PROMPT).render(application_description=application_description)
message = {"role": "user", "content": content}

In [5]:
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    output = typespec.TypespecTaskNode.run([message])

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [6]:
output.output.feedback

{'exit_code': 1,
 'stdout': "TypeSpec compiler v0.64.0\n\nDiagnostics were reported during compilation:\n\nschema.tsp:9:18 - error token-expected: '}' expected.\n>  9 |     make: string;\n     |                  ^\n> 10 |     model: string;\n     | ^\nschema.tsp:10:10 - error token-expected: Identifier expected.\n> 10 |     model: string;\n     |          ^\n\nFound 2 errors.\n\n",
 'stderr': None}

In [7]:
root = typespec.TypespecTaskNode(output)

In [8]:
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    solution = common.bfs(root)

In [9]:
solution.score, solution.depth

(1, 1)

In [10]:
for node in solution.parent.children:
    print(node.is_successful, node.data.output)

False TypespecOutput(reasoning="The original TypeSpec had correct structure but was missing semicolons after property definitions. In TypeSpec, property definitions within models must end with semicolons.\n\nI'll review the code for:\n1. Adding missing semicolons\n2. Checking for reserved keywords (none found in this case)\n3. Maintaining proper structure and indentation\n\nAll model names (PoemStyle, CarDetails, Poem) and property names (name, structure, make, model, etc.) are valid and not reserved keywords.", typespec_definitions='model PoemStyle {\n    name: string;  // e.g., "sonnet", "haiku", "limerick", "free verse"\n    structure: string;  // brief description of the form\n}\n\nmodel CarDetails {\n    make: string;\n    model: string;\n    characteristics: string[];  // e.g., ["fast", "luxurious", "electric"]\n}\n\nmodel Poem {\n    car: CarDetails;\n    style: PoemStyle;\n    content: string;\n    created: utcDateTime;\n}\n\ninterface CarPoetBot {\n    @llm_func(2)\n    create

In [11]:
with typespec.TypespecTaskNode.platform(client, compiler, jinja_env):
    args = root.run_args

In [12]:
args

[{'role': 'assistant',
  'content': '<reasoning>\nFor a bot that writes poems about cars, I expect users to send messages like "Write a poem about Tesla" or "Make a limerick about Ferrari".\nThe LLM needs to understand:\n1. The car make/model to write about\n2. The style of poem requested (if specified)\n3. Optional characteristics to emphasize (speed, luxury, etc.)\n\nThe main function would be createPoem which takes these parameters.\nWe also might want to save favorite poems and list them, so we need a Poem model with metadata.\n</reasoning>\n\n<typespec>\nmodel PoemStyle {\n    name: string;  // e.g., "sonnet", "haiku", "limerick", "free verse"\n    structure: string;  // brief description of the form\n}\n\nmodel CarDetails {\n    make: string;\n    model: string;\n    characteristics: string[];  // e.g., ["fast", "luxurious", "electric"]\n}\n\nmodel Poem {\n    car: CarDetails;\n    style: PoemStyle;\n    content: string;\n    created: utcDateTime;\n}\n\ninterface CarPoetBot {\n  